## Movie Recommender System Using AutoEncoders In PyTorch

### Movie rating is between 1 and 5 stars

In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
# Importing the dataset
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [28]:
# Preparing the training set and the test set
#training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = pd.read_csv('ml-1m/training_set.csv', sep = ',')
print(training_set.head())
training_set = np.array(training_set, dtype = 'int')
#test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = pd.read_csv('ml-1m/test_set.csv', sep = ',')
print('\n',test_set.head())
test_set = np.array(test_set, dtype = 'int')

   User  Movie  Rating  Timestamp
0     1    661       3  978302109
1     1    914       3  978301968
2     1   3408       4  978300275
3     1   2355       5  978824291
4     1   1287       5  978302039

    User  Movie  Rating  Timestamp
0     1   1193       5  978300760
1     1   1197       3  978302268
2     1   2804       5  978300719
3     1    595       5  978824268
4     1    938       4  978301752


In [29]:
# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [30]:
# Converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [31]:
# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [32]:
# Creating the architecture of the Neural Network
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [33]:
# Training the SAE
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data[0]*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

C:\Users\AK\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:16: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  app.launch_new_instance()


epoch: 1 loss: tensor(1.3478)
epoch: 2 loss: tensor(1.0102)
epoch: 3 loss: tensor(0.9900)
epoch: 4 loss: tensor(0.9833)
epoch: 5 loss: tensor(0.9801)
epoch: 6 loss: tensor(0.9786)
epoch: 7 loss: tensor(0.9773)
epoch: 8 loss: tensor(0.9765)
epoch: 9 loss: tensor(0.9760)
epoch: 10 loss: tensor(0.9757)
epoch: 11 loss: tensor(0.9752)
epoch: 12 loss: tensor(0.9751)
epoch: 13 loss: tensor(0.9747)
epoch: 14 loss: tensor(0.9746)
epoch: 15 loss: tensor(0.9744)
epoch: 16 loss: tensor(0.9743)
epoch: 17 loss: tensor(0.9743)
epoch: 18 loss: tensor(0.9741)
epoch: 19 loss: tensor(0.9740)
epoch: 20 loss: tensor(0.9739)
epoch: 21 loss: tensor(0.9738)
epoch: 22 loss: tensor(0.9737)
epoch: 23 loss: tensor(0.9736)
epoch: 24 loss: tensor(0.9733)
epoch: 25 loss: tensor(0.9728)
epoch: 26 loss: tensor(0.9722)
epoch: 27 loss: tensor(0.9715)
epoch: 28 loss: tensor(0.9707)
epoch: 29 loss: tensor(0.9702)
epoch: 30 loss: tensor(0.9693)
epoch: 31 loss: tensor(0.9683)
epoch: 32 loss: tensor(0.9673)
epoch: 33 loss: t

In [34]:
# Testing the SAE
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data[0]*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

C:\Users\AK\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:13: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  del sys.path[0]


test loss: tensor(0.8965)


### The final test set loss is less that one star